In [2]:
import pandas as pd
import numpy as numpy
import os

In [3]:
data = pd.read_csv('injuries_2024.csv')

In [4]:
sorted = data.sort_values(by=['Player', 'GameAdj'])

In [5]:
injuries = []
prev_row = None

for idx, row in sorted.iterrows():
    if prev_row is None:
        injuries.append(row)
    else:
        new_player = row['Player'] != prev_row['Player']
        new_injury_type = row['Injury Type'] != prev_row['Injury Type']
        gameadj_jump = row['GameAdj'] - prev_row['GameAdj']

        if new_player:
            injuries.append(row)
        elif new_injury_type:
            injuries.append(row)
        elif gameadj_jump > 1:
            injuries.append(row)
    prev_row = row

injury_df = pd.DataFrame(injuries)



In [8]:
injury_df = injury_df.sort_values(by=['GameAdj', 'Player'])

In [181]:
adj_2024_start = injury_df['GameAdj'].max()
adj_2025_start = injury_df['GameAdj'].max() + 82

In [ ]:
# First do the 2024-25 season

files = [
    f for f in os.listdir('Team Data/2024')
    if f.endswith('.csv')
]
team_data = []

for file in files:
    prev_row = None
    team_file = pd.read_csv(f'Team Data/2024/{file}', encoding='utf-16-le', sep='\t', engine='python')
    team_file = team_file.sort_values(by=['Player', 'Game'])
    team_file['GameAdj'] = team_file['Game'] + adj_2024_start
    split = file.split('_')
    team_label = split[0]
    team_file['Team'] = team_label
    team_file['Season'] = '2024/25'
    

    for idx, row in team_file.iterrows():
        if prev_row is None:
            team_data.append(row)
        else:
            new_player = row['Player'] != prev_row['Player']
            new_injury_type = row['Injury'] != prev_row['Injury']
            gameadj_jump = row['GameAdj'] - prev_row['GameAdj']

            if new_player:
                team_data.append(row)
            elif new_injury_type:
                team_data.append(row)
            elif gameadj_jump > 1:
                team_data.append(row)
        prev_row = row

# Now do 2025/26

files = [
    f for f in os.listdir('Team Data/2025')
    if f.endswith('.csv')
]

for file in files:
    prev_row = None
    team_file = pd.read_csv(f'Team Data/2025/{file}', encoding='utf-16-le', sep='\t', engine='python')
    team_file = team_file.sort_values(by=['Player','Game'])
    team_file['GameAdj'] = team_file['Game'] + adj_2025_start
    split = file.split('_')
    team_label = split[0]
    team_file['Team'] = team_label
    team_file['Season'] = '2025/26'
    

    for idx, row in team_file.iterrows():
        if prev_row is None:
            team_data.append(row)
        else:
            new_player = row['Player'] != prev_row['Player']
            new_injury_type = row['Injury'] != prev_row['Injury']
            gameadj_jump = row['GameAdj'] - prev_row['GameAdj']

            if new_player:
                team_data.append(row)
            elif new_injury_type:
                team_data.append(row)
            elif gameadj_jump > 1:
                team_data.append(row)
        prev_row = row

team_injuries = pd.DataFrame(team_data)

In [219]:
team_injuries = team_injuries.drop(columns=['Position (copy)'])
team_injuries = team_injuries.rename(columns={'Injury':'Injury Type'})

In [221]:
combined = pd.concat(objs=[injury_df, team_injuries], join='inner')

In [222]:
combined.sort_values(by=['GameAdj', 'Player'])

,Player,GameAdj,Game,Injury Type,Position,Season,Team
0,"Antropov, Nik",1,1,Knee,F,2000/01,Toronto
1,"Bergevin, Marc",1,1,Thumb,D,2000/01,St. Louis
2,"Brown, Doug",1,1,Knee,F,2000/01,Detroit
3,"Burt, Adam",1,1,Cheek,D,2000/01,Atlanta
4,"Cairns, Eric",1,1,Thumb,D,2000/01,NY Islanders
...,...,...,...,...,...,...,...
86,Middleton,1928,31,Upper body,D,2025/26,Minnesota
123,Tuch,1928,31,Illness,F,2025/26,Buffalo
37,Zuccarello,1928,31,Upper body,F,2025/26,Minnesota
84,Bjugstad,1929,32,Upper body,F,2025/26,St. Louis


In [232]:
combined.to_csv('full_data.csv', index=False)